# 크롤러 & csv 저장

## 세팅

In [1]:
!pip install beautifulsoup4

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from google.colab import drive

In [3]:
drive.mount('gdrive')
save_path ='gdrive/My Drive/BD/naver_news_crawling/'

Drive already mounted at gdrive; to attempt to forcibly remount, call drive.mount("gdrive", force_remount=True).


## 함수정의

In [4]:
def crawler(save_path, max_page, query, sort, start_date, end_date):
  page = 1
  number_news = (int(max_page)-1)*10+1
  s_from = start_date.replace(".","")
  e_to = end_date.replace(".","")

  while page <= number_news:
    url = "http://search.naver.com/search.naver?where=news&query=" + query + "&sort=" + str(sort) + "&ds=" + start_date + \
            "&de=" + end_date+ "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)

    response = requests.get(url)
    html = response.text

    soup = BeautifulSoup(html, 'html.parser')

    atags = soup.select('.news_tit')
    for atag in atags:
      title_text.append(atag.text)
      link_text.append(atag['href'])

    source_lists = soup.select('.info_group > .press')
    for source_list in source_lists:
      source_text.append(source_list.text)

    date_lists = soup.select('.info_group > span.info')
    for date_list in date_lists:
      if date_list.text.find("면") == -1:
        date_text.append(date_list.text)

    contents_lists = soup.select('.news_dsc')
    for contents_list in contents_lists:
      contents_cleansing(contents_list)

    result= {"date" : date_text , "title":title_text , "source" : source_text ,"contents": contents_text ,"link":link_text }
    print(page)
    df = pd.DataFrame(result)
    page += 10

  outputFileName = '%s 관련 뉴스 %s ~ %s.csv' % (query, start_date, end_date)
  df.to_csv(save_path+outputFileName)


In [5]:
def date_cleansing(test):
    try:
        pattern = '\d+.(\d+).(\d+).'
    
        r = re.compile(pattern)
        match = r.search(test).group(0)
        date_text.append(match)
        
    except AttributeError:
        pattern = '\w* (\d\w*)'
        
        r = re.compile(pattern)
        match = r.search(test).group(1)
        date_text.append(match)

In [6]:
def contents_cleansing(contents):
  first_cleansing_contents = re.sub('<dl>.*?</a> </div> </dd> <dd>', '',
  str(contents)).strip()
  second_cleansing_contents = re.sub('<ul class="relation_lst">.*?</dd>', '',
  first_cleansing_contents).strip()
  third_cleansing_contents = re.sub('<.+?>', '', second_cleansing_contents).strip()
  contents_text.append(third_cleansing_contents)

## 실행

In [7]:
title_text= []
link_text= []
source_text= []
date_text= []
contents_text = []
result = {}

In [8]:
max_page = 10
query = "주식"  # 검색어
sort = 0  #  0:관련도순  /  1:최신순  /  2:오래된순
start_date = "20220202"  # 시작 날짜
end_date = "20220303" # 끝 날짜

In [9]:
crawler(save_path, max_page, query, sort, start_date, end_date)

1
11
21
31
41
51
61
71
81
91


In [10]:
data_path = 'gdrive/My Drive/BD/naver_news_crawling/'
filename = query + ' 관련 뉴스 ' + start_date + ' ~ ' + end_date + '.csv'

In [11]:
df = pd.read_csv(data_path + filename)

In [12]:
df.head()

,Unnamed: 0,date,title,source,contents,link
0,0,2022.03.03.,"이재명 ""부동산 한물 갔다…제 당선 믿으면 자산 증식은 주식으로""",아시아경제언론사 선정,"아니고 주식시장으로, 자본시장으로 가라""고 밝혔다. 이 후보는 이날 오후 서울 강서...",https://view.asiae.co.kr/article/2022030320264...
1,1,2022.03.03.,"러시아 ETF 투자자들 어쩌나…MSCI ""러 주식 0.00001 가격 적용""",한국경제언론사 선정,모건스탠리인터내셔널(MSCI) 측이 모든 지수 내 러시아 주식에 대해 0.00001...,https://www.hankyung.com/finance/article/20220...
2,2,2022.03.03.,"신세계百, 계열사 신세계라이브쇼핑 주식 전량 인수…디지털전환 박차(종합)",연합뉴스,"이마트 지분 47.8%, 신세계I&amp;C 지분 28.3% 총 2천255억원에 취...",http://yna.kr/AKR20220303147051002?did=1195m
3,3,2022.03.03.,"신세계, 신세계라이브쇼핑 주식 1126만주 취득 [증시이슈]",이코노미스트언론사 선정,‘신세계라이브쇼핑’의 주식 1126만주를 취득한다고 3일 공시했다. 이마트(47.8...,https://economist.co.kr/2022/03/03/stock/stock...
4,4,2022.03.02.,"국민카드, 'KB Pay'로 소액투자 가능한 해외주식투자 서비스 개시",뉴스1,KB국민카드가 한국투자증권과 함께 'KB페이'(KB Pay)를 통해 소액으로 환전 ...,https://www.news1.kr/articles/?4601635
